# F-classify-train: This Notebook trains the NOIRE-Net F-region classification networks

## 1 - Import libaries 

In [1]:
import os
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.metrics import mean_squared_error
from math import sqrt
from tensorflow.keras.callbacks import ModelCheckpoint
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.metrics import cohen_kappa_score, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

## 2 - Define function to get ionogram label (True = F-region, False = No F-region)

In [2]:
# The function get_label_from_par reads a .par file and returns 
# True if either the second or fourth item is not 'nan', 
# indicating the presence of an F-region in the corresponding image,
# for use in binary classification.

def get_label_from_par(par_file_path):
    try:
        # Open the .par file located at par_file_path
        with open(par_file_path, 'r') as file:
            content = file.readline().strip()  # Read and strip the first line of the file
            items = content.split()  # Split the line into individual items (usually numbers or 'nan')

            # Check the first (index 0) and third (index 2) items:
            # If either of these items is not 'nan', it implies the presence of an F-region.
            # The function then returns True, indicating that this image has an F-region.
            is_f_region = items[0].lower() != 'nan' or items[2].lower() != 'nan'
        return is_f_region

    except IndexError:
        # This block catches an IndexError, which occurs if the line read from the file
        # does not have at least four items. In such a case, the function assumes that
        # the F-region data is not present or not properly formatted, and returns False.
        return False

    except Exception as e:
        # This block catches any other exceptions (like file not found, read errors, etc.).
        # It prints the exception message and returns False, indicating an issue with processing the file.
        print(f"Error reading {par_file_path}: {e}")
        return False

## 3 - Define function to load ionograms and preprocess the data

In [3]:
# The load_data function loads and preprocesses image data from a specified directory,
# converting images to grayscale and resizing them, while also extracting corresponding
# binary labels from associated .par files for a classification task.

def load_data(data_dir):
    images = []  # List to store preprocessed images
    labels = []  # List to store corresponding labels

    # Construct paths to the directories containing ionograms and parameters
    ionograms_dir = os.path.join(data_dir, 'ionograms')
    parameters_dir = os.path.join(data_dir, 'parameters')

    # Iterate over the files in the ionograms directory
    for filename in os.listdir(ionograms_dir):
        if filename.endswith('.png'):  # Check if the file is a PNG image
            # Construct full paths to the image file and its corresponding .par file
            img_path = os.path.join(ionograms_dir, filename)
            par_path = os.path.join(parameters_dir, filename.replace('.png', '.par'))

            # Load the image, convert it to grayscale, resize it, and normalize pixel values
            image = load_img(img_path, color_mode='grayscale', target_size=(310, 310))
            image = img_to_array(image)
            image /= 255.0  # Normalize image pixels to be between 0 and 1

            # Load the label for the image using the get_label_from_par function
            label = get_label_from_par(par_path)

            # Append the preprocessed image and label to their respective lists
            images.append(image)
            labels.append(label)

    # Convert the lists of images and labels to numpy arrays and return them
    return np.array(images), np.array(labels).astype(int)

## 4 - Load the ionograms and labels from the data folder 

In [4]:
# Specify the directory where the data is stored
data_dir = 'train-val-test'  # 'train_test_val' should be replaced with the actual path to your data directory

# Call the load_data function to load and preprocess the data
# X wildsdsdl contain the preprocessed images, and y will contain the corresponding labels
X, y = load_data(data_dir)

## 5 - Define a function to create the NOIRE-Net architecture

In [5]:
# This code defines and complies NOIRE-Net a convolutional neural network (CNN) model using Keras, 
# with multiple convolutional layers, batch normalization, max pooling, and dense layers, 
# designed for binary classification tasks.

def NOIREnet():
    model = Sequential([
    # First convolutional layer with 32 filters and a kernel size of 3x3
    # 'padding=same' ensures the output size is the same as the input size
    # 'input_shape' is set for the first layer to indicate the shape of the input data
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(310, 310, 1)),
    
    # Batch normalization to normalize the activations from the previous layer
    BatchNormalization(),

    # Second convolutional layer with 32 filters and a kernel size of 3x3
    Conv2D(32, (3, 3), activation='relu'),

    # Another batch normalization
    BatchNormalization(),

    # First max pooling layer to reduce spatial dimensions
    MaxPooling2D((2, 2)),

    # Repeating the pattern of two convolutional layers followed by batch normalization
    # and a max pooling layer, gradually increasing the number of filters
    Conv2D(32, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Flatten the output from the convolutional layers to feed into dense layers
    Flatten(),

    # Dense (fully connected) layer with 256 neurons and relu activation
    Dense(256, activation='relu'),

    # Dropout layer to reduce overfitting
    Dropout(0.5),

    # Another dense layer with 128 neurons
    Dense(128, activation='relu'),

    # Output layer with a single neuron and sigmoid activation for binary classification
    Dense(1, activation='sigmoid')
    ])
    
    # Compile the CNN model
    model.compile(
        optimizer='adam',  # Using the Adam optimizer for adaptive learning rate optimization
        loss='binary_crossentropy',  # Binary crossentropy loss function, suitable for binary classification tasks
        metrics=['accuracy']  # The model will report 'accuracy' as a performance metric
    )
    
    # Return the compiled model
    return model

## 6 - Train 10 CNNs for F-region classification and save the models

In [6]:
# This code trains 10 Convolutional Neural Networks (CNNs) on differently split subsets
# of a dataset for binary classification,saves the best model of each training session, 
# and records their training histories.

import os
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Create the directory for saving models and histories if it doesn't exist
save_dir = 'F-classify'
os.makedirs(save_dir, exist_ok=True)

# Initialize lists to store the training histories and filenames of the best models
histories = []
model_filenames = []

# Define the ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.1,      # Factor to reduce the learning rate
    patience=20,     # Number of epochs with no improvement to wait before reducing LR
    min_lr=0.00001   # Minimum learning rate
)

# Loop to train 10 CNN models with different data splits
for i in range(10):
    # Split the dataset into training, validation and testing sets with stratification
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=i)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=i)
    
    # Create a new CNN model for each iteration
    model = NOIREnet()

    # Define the filename for the checkpoint model
    model_filename = os.path.join(save_dir, f'F-classify_run{i+1}.h5')

    # Define a checkpoint callback to save the best model based on validation accuracy
    checkpoint_callback = ModelCheckpoint(
        model_filename,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='max',
        save_weights_only=False
    )

    # Train the model with specified callbacks including ReduceLROnPlateau
    history = model.fit(
        X_train, y_train,
        batch_size=64,
        epochs=100,
        validation_data=(X_val, y_val),
        callbacks=[checkpoint_callback, reduce_lr]  # Include ReduceLROnPlateau callback
    )

    # Save the training history and the filename of the saved best model
    histories.append(history.history)
    model_filenames.append(model_filename)

# Optionally, save the training histories to a file in the same 'F-classify' directory
history_filename = os.path.join(save_dir, 'training_histories.pkl')
with open(history_filename, 'wb') as file:
    pickle.dump({'histories': histories, 'model_filenames': model_filenames}, file)


Metal device set to: Apple M1 Max


2023-12-05 21:29:30.590054: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-05 21:29:30.590286: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/100


2023-12-05 21:29:34.276848: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-12-05 21:29:35.777070: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x64x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be

183/184 [============================>.] - ETA: 0s - loss: 0.7446 - accuracy: 0.5118

...


Epoch 1: val_accuracy improved from -inf to 0.50636, saving model to F-classify/F-classify_run10.h5


loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x20x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x20x1x1xi1>'


184/184 [==============================] - 67s 346ms/step - loss: 0.7288 - accuracy: 0.5159 - val_loss: 0.7038 - val_accuracy: 0.5064 - lr: 0.0010
Epoch 2/100
184/184 [==============================] - ETA: 0s - loss: 0.6879 - accuracy: 0.5208
Epoch 2: val_accuracy did not improve from 0.50636
184/184 [==============================] - 63s 341ms/step - loss: 0.6879 - accuracy: 0.5208 - val_loss: 1.4163 - val_accuracy: 0.4936 - lr: 0.0010
Epoch 3/100
184/184 [==============================] - ETA: 0s - loss: 0.6850 - accuracy: 0.5124
Epoch 3: val_accuracy improved from 0.50636 to 0.89547, saving model to F-classify/F-classify_run10.h5
184/184 [==============================] - 64s 346ms/step - loss: 0.6850 - accuracy: 0.5124 - val_loss: 0.3197 - val_accuracy: 0.8955 - lr: 0.0010
Epoch 4/100
184/184 [==============================] - ETA: 0s - loss: 0.6788 - accuracy: 0.5170
Epoch 4: val_accuracy improved from 0.89547 to 0.90382, saving model to F-classify/F-classify_run10.h5
184/184 [==

Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG13XFamilyCommandBuffer: 0x310e43620>
    label = <none> 
    device = <AGXG13XDevice: 0x153301c00>
        name = Apple M1 Max 
    commandQueue = <AGXG13XFamilyCommandQueue: 0x153596000>
        label = <none> 
        device = <AGXG13XDevice: 0x153301c00>
            name = Apple M1 Max 
    retainedReferences = 1


184/184 [==============================] - ETA: 0s - loss: 0.6602 - accuracy: 0.5314
Epoch 38: val_accuracy did not improve from 0.91296
184/184 [==============================] - 69s 377ms/step - loss: 0.6602 - accuracy: 0.5314 - val_loss: 0.4468 - val_accuracy: 0.9030 - lr: 1.0000e-04
Epoch 39/100
184/184 [==============================] - ETA: 0s - loss: 0.6598 - accuracy: 0.5315
Epoch 39: val_accuracy did not improve from 0.91296
184/184 [==============================] - 70s 382ms/step - loss: 0.6598 - accuracy: 0.5315 - val_loss: 0.4658 - val_accuracy: 0.9014 - lr: 1.0000e-04
Epoch 40/100
184/184 [==============================] - ETA: 0s - loss: 0.6597 - accuracy: 0.5316
Epoch 40: val_accuracy did not improve from 0.91296
184/184 [==============================] - 72s 390ms/step - loss: 0.6597 - accuracy: 0.5316 - val_loss: 0.5200 - val_accuracy: 0.9034 - lr: 1.0000e-04
Epoch 41/100
184/184 [==============================] - ETA: 0s - loss: 0.6596 - accuracy: 0.5316
Epoch 41: va